<a href="https://colab.research.google.com/github/Edsonalex/class-planner/blob/main/Plano_de_Aula.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install google-genai
!pip install -q pdfkit
!apt-get update -qq
!apt-get install -q wkhtmltopdf

In [115]:
# Configura a API Key do Google Gemini

import os
import google.generativeai as genai
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [116]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [117]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

In [118]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [119]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [120]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [121]:
##########################################
# --- Agente 1: Planejador Pedagógico --- #
##########################################
class PlanejadorPedagogico:
    def __init__(self, model):
        self.agente_conteudo = AgenteConteudo(model)
        self.agente_atividades = AgenteAtividades(model)
        self.agente_avaliacao = AgenteAvaliacao(model)
        self.agente_agenda = AgenteAgenda(model)

    def planejar_aula(self, serie, disciplina, tema, duracao):
        conteudo = self.agente_conteudo.sugerir_conteudo(serie, disciplina, tema)
        atividades = self.agente_atividades.sugerir_atividades(serie, tema)
        avaliacoes = self.agente_avaliacao.sugerir_avaliacoes(tema)
        cronograma = self.agente_agenda.gerar_cronograma(duracao, conteudo, atividades, avaliacoes)

        return {
            "Série": serie,
            "Disciplina": disciplina,
            "Tema": tema,
            "Conteúdo": conteudo,
            "Atividades": atividades,
            "Avaliações": avaliacoes,
            "Cronograma": cronograma
        }

In [136]:
################################################
# --- Agente 2: Criador de Conteúdo --- #
################################################
class AgenteConteudo:
    def __init__(self, model):
        self.model = model
    def sugerir_conteudo(self, serie, disciplina, tema):
        prompt = (
            f"Você é um especialista em educação básica. Sugira um conteúdo didático para uma aula de {disciplina} "
            f"do {serie} ano do ensino fundamental sobre o tema '{tema}', de acordo com a BNCC. Seja objetivo e sucinto. Escreva como um professor."
        )
        response = self.model.generate_content(prompt)
        return response.text

In [123]:
######################################
# --- Agente 3: Criador de Atividades --- #
######################################
class AgenteAtividades:
    def __init__(self, model):
        self.model = model
    def sugerir_atividades(self, serie, tema):
        prompt = (
            f"Sugira duas atividades lúdicas e práticas para alunos do {serie} ano do ensino fundamental, "
            f"sobre o tema '{tema}'. As atividades devem ser simples e possíveis de realizar em sala de aula."
        )
        response = self.model.generate_content(prompt)
        return response.text.split("\n")

In [124]:
##########################################
# --- Agente 4: Criador de Avaliações --- #
##########################################
class AgenteAvaliacao:
    def __init__(self, model):
        self.model = model
    def sugerir_avaliacoes(self, tema):
        prompt = (
            f"Sugira instrumentos de avaliação divididas nas cateorias Formativa (autoavaliação e observação) e Somativa(provas, testes, trabalhos) para o tema {tema} e a série {serie}.",
            f"Recomende avaliação escrita com problemas contextualizados."
        )
        response = self.model.generate_content(prompt)
        return response.text.split("\n")

In [125]:
##########################################
# --- Agente 5: Criador de Agenda --- #
##########################################
class AgenteAgenda:
    def __init__(self, model):
      self.model = model
    def gerar_cronograma(self, duracao, conteudo, atividades, avaliacoes):
        prompt = (
            f"Organize os conteúdos, atividades e avaliações num cronograma semanal em formato de calendário considerando o tema{tema}, a duração {duracao}, as atividades propostas {atividades} e as avalaliações {avaliacoes}.",
            f"Considere número de 4 aulas por semana e o tempo de 50 minutos necessário por aula"
        )
        response = self.model.generate_content(prompt)
        return response.text.split("\n")

In [137]:
def iniciar_prompt():
    print("=== Gerador de Plano de Aula ===")
    serie = input("Série (ex: 5º): ")
    disciplina = input("Disciplina (ex: Matemática): ")
    tema = input("Tema (ex: Frações): ")
    duracao = int(input("Duração em semanas: "))
    return serie, disciplina, tema, duracao

import google.generativeai as genai

model = genai.GenerativeModel(MODEL_ID)
planejador = PlanejadorPedagogico(model)

serie, disciplina, tema, duracao = iniciar_prompt()

plano = planejador.planejar_aula(serie, disciplina, tema, duracao)

print("\n\n🚀 Iniciando o Sistema de Criação de Planejamento Pedagógico 🚀")
print(f"Maravilha! Vamos então criar o planejamento em {disciplina} para alunos da {serie} série com o tema {tema} e com duração de {duracao} semana(s).")

conteudo = plano["Conteúdo"]
print("\n--- 📝 Resultado do Gerador de Conteúdo (Conteúdo) ---\n")
display(to_markdown(conteudo))
print("--------------------------------------------------------------")

atividades = plano["Atividades"]
print("\n--- 📝 Resultado do Gerador de Atividades (Atividades) ---\n")
for atividade in atividades:
    display(to_markdown(atividade))
print("--------------------------------------------------------------")
print("--------------------------------------------------------------")

avaliacoes = plano["Avaliações"]
print("\n--- 📝 Resultado do Gerador de Avaliações (Avaliações) ---\n")
for avaliacao in avaliacoes:
    display(to_markdown(avaliacao))
print("--------------------------------------------------------------")

cronograma = plano["Cronograma"]
print("\n--- 📝 Resultado do Gerador de Cronograma (Cronograma) ---\n")
if isinstance(cronograma, list):
    for item in cronograma:
        display(to_markdown(item))
elif isinstance(cronograma, dict):
    for key, value in cronograma.items():
        display(to_markdown(f"{key}:\n{value}"))
else:
    display(to_markdown(str(cronograma)))

print("--------------------------------------------------------------")

=== Gerador de Plano de Aula ===
Série (ex: 5º): 1
Disciplina (ex: Matemática): portugues
Tema (ex: Frações): alfabeto
Duração em semanas: 4


🚀 Iniciando o Sistema de Criação de Planejamento Pedagógico 🚀
Maravilha! Vamos então criar o planejamento em portugues para alunos da 1 série com o tema alfabeto e com duração de 4 semana(s).

--- 📝 Resultado do Gerador de Conteúdo (Conteúdo) ---



> ## Plano de Aula: Desvendando o Alfabeto (1º Ano)
> 
> **Objetivo Geral:** Apresentar o alfabeto como um conjunto de símbolos que representam os sons da fala, capacitando os alunos a reconhecerem e nomearem as letras.
> 
> **Objetivos Específicos (BNCC):**
> 
> *   **(EF01LP01)** Reconhecer que textos são lidos e escritos da esquerda para a direita e de cima para baixo, e que espaços em branco indicam separação entre palavras. (Introdução)
> *   **(EF01LP02)** Escrever, espontaneamente ou por ditado, palavras e frases de forma alfabética – usando letras/grafemas – para representar palavras e ideias. (Prática)
> *   **(EF01LP05)** Reconhecer o sistema de escrita alfabética como representação dos sons da fala. (Conceito)
> 
> **Recursos:**
> 
> *   Cartazes com o alfabeto (maiúsculo e minúsculo)
> *   Fichas com letras (individuais e conjuntos)
> *   Objetos/imagens cujos nomes iniciem com as letras do alfabeto
> *   Folhas de atividades com exercícios de identificação e escrita
> *   Música do alfabeto
> 
> **Desenvolvimento:**
> 
> 1.  **Roda de Conversa (10 min):** Iniciar a aula perguntando o que é o alfabeto, onde o encontramos e para que serve. Introduzir a ideia de que as letras são "desenhos" que representam os sons que falamos. (EF01LP01)
> 2.  **Apresentação do Alfabeto (15 min):** Apresentar o cartaz do alfabeto, nomeando cada letra e sua forma maiúscula e minúscula. Cantar a música do alfabeto.
> 3.  **Exploração Lúdica (20 min):**
>     *   **Caça às letras:** Espalhar fichas com letras pela sala e pedir que os alunos encontrem letras específicas.
>     *   **Letra inicial:** Mostrar objetos/imagens e pedir que os alunos identifiquem a letra inicial.
>     *   **Montando palavras:** Utilizar as fichas de letras para formar palavras simples, como "bola", "dado", etc.
> 4.  **Atividade Individual (20 min):** Distribuir folhas de atividades com exercícios de:
>     *   Identificação de letras (circular, colorir).
>     *   Correspondência letra-imagem.
>     *   Escrita das letras (EF01LP02, EF01LP05).
> 5.  **Compartilhamento e Correção (10 min):** Corrigir as atividades em conjunto, reforçando o aprendizado e tirando dúvidas.
> 
> **Avaliação:** Observação da participação dos alunos nas atividades, análise das produções escritas e identificação das letras do alfabeto.
> 
> **Observação:** Adapte as atividades ao ritmo e às necessidades da sua turma. Incentive a participação de todos e valorize o esforço individual.


--------------------------------------------------------------

--- 📝 Resultado do Gerador de Atividades (Atividades) ---



> Com certeza! Aqui estão algumas sugestões de atividades lúdicas e práticas sobre o alfabeto, adequadas para alunos do 1º ano do Ensino Fundamental, fáceis de implementar em sala de aula:

> **1. Caça ao Tesouro do Alfabeto:**

...

> *   **Comunicação:** Manter uma comunicação aberta com os pais, informando sobre o progresso dos alunos e solicitando apoio em casa.

> *   **Diversão:** Priorizar atividades lúdicas e motivadoras, tornando o aprendizado do alfabeto uma experiência prazerosa para os alunos.

> *   **Avaliação Contínua:** A avaliação deve ser contínua e diversificada, utilizando diferentes instrumentos e estratégias para acompanhar o progresso dos alunos.

> Este cronograma visa fornecer uma estrutura para o ensino do alfabeto, combinando atividades lúdicas, práticas e avaliações formativas e somativas para garantir um aprendizado eficaz e significativo.

--------------------------------------------------------------


In [138]:
'''
# --- Exportação para PDF (com Formatação CSS) ---
import pdfkit
import html
import os
from google.colab import files

def exportar_para_pdf_formatado(plano, nome_arquivo="plano_de_aula.pdf"):
    html_content = """
    <html>
    <head>
        <meta charset='UTF-8'>
        <title>Plano de Aula Formatado</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                line-height: 1.6;
                margin: 20mm; /* Adiciona margens nas páginas */
            }
            h1 {
                color: #333;
                text-align: center;
                margin-bottom: 20px;
            }
            h2 {
                color: #555;
                margin-top: 30px;
                border-bottom: 1px solid #eee;
                padding-bottom: 5px;
            }
            h3 {
                color: #777;
                margin-top: 20px;
                margin-bottom: 10px;
            }
            ul {
                margin-bottom: 20px;
            }
            li {
                margin-bottom: 5px;
            }
            p {
                margin-bottom: 15px;
            }
            /* Exemplo: Adicionar espaço entre as seções */
            .section {
                margin-bottom: 40px;
            }
        </style>
    </head>
    <body>
    <h1>Plano de Aula</h1>
    """

    for chave, valor in plano.items():
        html_content += f"<div class='section'>"
        html_content += f"<h2>{html.escape(str(chave))}</h2>"
        if isinstance(valor, list):
            html_content += "<ul>"
            for item in valor:
                item_limpo = item.replace('•', '').strip()
                html_content += f"<li>{html.escape(str(item_limpo))}</li>"
            html_content += "</ul>"
        elif isinstance(valor, dict):
            for subchave, itens in valor.items():
                html_content += f"<h3>{html.escape(str(subchave))}</h3><ul>"
                if isinstance(itens, list):
                     for item in itens:
                        item_limpo = item.replace('•', '').strip()
                        html_content += f"<li>{html.escape(str(item_limpo))}</li>"
                     html_content += "</ul>"
                else:
                     html_content += f"<p>{html.escape(str(itens))}</p>"
        else:
            html_content += f"<p>{html.escape(str(valor))}</p>"
        html_content += "</div>"

    html_content += "</body></html>"

    html_temp_file = "temp_plano_de_aula_formatado.html"
    try:
        with open(html_temp_file, "w", encoding="utf-8") as f:
            f.write(html_content)

        path_wkhtmltopdf = r'/usr/bin/wkhtmltopdf'
        config = pdfkit.configuration(wkhtmltopdf=path_wkhtmltopdf)

        # Converter o arquivo HTML temporário para PDF
        pdfkit.from_file(html_temp_file, nome_arquivo, configuration=config)
        print(f"\nPDF formatado exportado como '{nome_arquivo}'")

        # Arquivo para download
        files.download(nome_arquivo)

    except Exception as e:
        print(f"Ocorreu um erro ao exportar para PDF formatado: {e}")
        print("Certifique-se de que wkhtmltopdf está instalado corretamente e acessível no caminho especificado.")

    finally:
        # Remover HTML temporário
        if os.path.exists(html_temp_file):
            os.remove(html_temp_file)

exportar_para_pdf_formatado(plano)
'''

'\n# --- Exportação para PDF (com Formatação CSS) ---\nimport pdfkit\nimport html\nimport os\nfrom google.colab import files\n\ndef exportar_para_pdf_formatado(plano, nome_arquivo="plano_de_aula.pdf"):\n    html_content = """\n    <html>\n    <head>\n        <meta charset=\'UTF-8\'>\n        <title>Plano de Aula Formatado</title>\n        <style>\n            body {\n                font-family: Arial, sans-serif;\n                line-height: 1.6;\n                margin: 20mm; /* Adiciona margens nas páginas */\n            }\n            h1 {\n                color: #333;\n                text-align: center;\n                margin-bottom: 20px;\n            }\n            h2 {\n                color: #555;\n                margin-top: 30px;\n                border-bottom: 1px solid #eee;\n                padding-bottom: 5px;\n            }\n            h3 {\n                color: #777;\n                margin-top: 20px;\n                margin-bottom: 10px;\n            }\n       